# Zambon - List

## Beachten
* Kein PDF vorhanden. Zambon bietet nur eine Website. Vorgehen:
 * Website öffnen: http://www.zambon-ch.ch/de/disclosure/
 * Kompletter Quellcode der Website (`Rechtsklick` -> `Seite Speichern unter`) rauskopieren und in Datei `website_dump.html` speichern
* Type manuell hinzufügen

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import copy
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import HTML with Beautiful Soup

In [2]:
with open("website_dump.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')
    
tbl = soup.find("table", class_="sheet0")

In [3]:
record_list = []
counter = 0
for row in tbl.find_all("tr", recursive=True):
    #Skip header
    counter += 1
    if counter < 6:
        continue
       
    #Skip if not exactle 13 cells
    if len(row.find_all('td')) != 13:
        continue

    record = {}
    record['name'] = row.find('td', class_='column2').text
    record['location'] = row.find('td', class_='column3').text
    record['country'] = row.find('td', class_='column4').text
    record['address'] = row.find('td', class_='column5').text
    record['uci'] = row.find('td', class_='column6').text
    record['donations_grants'] = row.find('td', class_='column7').text
    record['sponsorship'] = row.find('td', class_='column8').text
    record['registration_fees'] = row.find('td', class_='column9').text
    record['travel_accommodation'] = row.find('td', class_='column10').text
    record['fees'] = row.find('td', class_='column11').text
    record['related_expenses'] = row.find('td', class_='column12').text
    record['total'] = row.find('td', class_='column14').text

    record_list.append(record)

In [4]:
#Load Dataframe
df = pd.DataFrame(record_list)

## Format Table

In [5]:
df_export = df.copy()

#Remove rows without values
df_export = df_export[df_export['country'] != '']

#Add missing fields
df_export.insert(0, 'plz', '')

#Add Type
index_hco = df_export[df_export['name'].str.contains("Parkinson Schweiz", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')
add_warning(manually=True)

#Convert to Numbers & Sum
df_export = amounts_to_number(df_export)

#Reorder columns
df_export = df_export[fix_columns[:-1]]

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'zambon')

saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx')